In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

In [0]:
# read from Bronz table 
df = spark.table("workspace.bronze.erp_px_cat_g1v2")
df.show(5)

In [0]:
# triming 
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
    df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
%sql 
DESCRIBE workspace.bronze.erp_px_cat_g1v2

Normalize maintainance

In [0]:
df = df.withColumn("MAINTENANCE", F.when(F.upper(col("MAINTENANCE")) == "YES", F.lit(True))
.when(F.upper(col("MAINTENANCE")) == "NO", F.lit(False))
.otherwise(None)
)
df.show(5)


In [0]:
# Rename the columns
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "MAINTENANCE": "Maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     

In [0]:
# sanity check 
df.display()

In [0]:
# write to the silver table 
df.write.mode("overwrite").saveAsTable("workspace.silver.erp_product_category")

In [0]:
%sql
select * from workspace.silver.erp_product_category limit 5